In [ ]:


from sklearn.datasets import make_blobs
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeClassifier
# define dataset
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score
from xgboost import XGBClassifier
from matplotlib import pyplot
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve, auc
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier

import lightgbm as lgb
       
import matplotlib.pyplot as plt
# from sklearn.metrics import r2_score
import re

# import matplotlib.pyplot as plt
# %matplotlib inline


random_seed = 5

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)
import re

# code for uploading pickled data

M_X_train_ready = pickle.load(open("Train_samples\M_X_train_ready.pkl", 'rb'))
M_Y_train_ready = pickle.load(open("Train_samples\M_Y_train.pkl", 'rb'))
M_X_train_ready = pd.concat(M_X_train_ready.values(), join='inner', ignore_index=True)
M_Y_train_ready = pd.concat(M_Y_train_ready.values(), join='inner', ignore_index=True)

F_X_train_ready = pickle.load(open("Train_samples\F_X_train_ready.pkl", 'rb'))
F_Y_train_ready = pickle.load(open("Train_samples\F_Y_train.pkl", 'rb'))
F_X_train_ready = pd.concat(F_X_train_ready.values(), join='inner', ignore_index=True)
F_Y_train_ready = pd.concat(F_Y_train_ready.values(), join='inner', ignore_index=True)

M_X_test_ready = pickle.load(open("Test_samples\M_X_test_ready.pkl", 'rb'))
M_Y_test_ready = pickle.load(open("Test_samples\M_Y_test.pkl", 'rb'))
M_X_test_ready = pd.concat(M_X_test_ready.values(), join='inner', ignore_index=True)
M_Y_test_ready = pd.concat(M_Y_test_ready.values(), join='inner', ignore_index=True)

F_X_test_ready = pickle.load(open("Test_samples\F_X_test_ready.pkl", 'rb'))
F_Y_test_ready = pickle.load(open("Test_samples\F_Y_test.pkl", 'rb'))
F_X_test_ready = pd.concat(F_X_test_ready.values(), join='inner', ignore_index=True)
F_Y_test_ready = pd.concat(F_Y_test_ready.values(), join='inner', ignore_index=True)



CM_test_ready = pickle.load(open("Left_one_out_samples\CM_test_ready.pkl", 'rb'))
CM_Y_test = pickle.load(open("Left_one_out_samples\CM_Y_test.pkl", 'rb'))
CM_test_ready = pd.concat(CM_test_ready.values(), join='inner', ignore_index=True)
CM_Y_test = pd.concat(CM_Y_test.values(), join='inner', ignore_index=True)


CF_test_ready = pickle.load(open("Left_one_out_samples\CF_test_ready.pkl", 'rb'))
CF_Y_test = pickle.load(open("Left_one_out_samples\CF_Y_test.pkl", 'rb'))
CF_test_ready = pd.concat(CF_test_ready.values(), join='inner', ignore_index=True)
CF_Y_test = pd.concat(CF_Y_test.values(), join='inner', ignore_index=True)


M_selected_features = ['Age', 'Relationship with family head','Respondent live in household',
                        'Known HIV status','Bought/sold sex in the past 12 months',
                        'Condom was used at last paid sex in past 12 months','LAg: recent/long term infection',
                        'CD4 category','on ART','ART initiated past 12 months those ever on ART',
                        'last sex partner relations past 12 months','Sick to work last three months',
                        'Ever attended school','Ever enrolled in school','Highest level of education',
                        'Highest grade at that school level', 'Work for payment in last 12 months',
                        'Ever married/lived together','Avoiding pregnancy','CFcumcision status',
                        'Age at fFst sex', 'Ever sought TB treatment','Alcohol drink frequency',
                        'Urban area indicator','Wealth quintile', 'Sexual intercourse in the past 12 months',
                        'Ever had sexual intercourse'] 
M_X_train_ready_selected = M_X_train_ready[M_selected_features]
F_selected_features = ['Age', 'Relationship with family head', 'Respondent live in household',
       'Sick to work last three months', 'Ever attended school',
       'Ever enrolled in school', 'Highest level of education',
       'Highest grade at that school level',
       'Work for payment in last 12 months', 'Ever married/lived together',
       'Number of pregnancies',
       'Ever had a pregnancy that resulted in a live bFth',
       'No. of childen given bFth since 2012', 'Pregnacy status now',
       'Avoiding pregnancy', 'Age at fFst sex', 'Ever sought TB treatment',
       'Alcohol drink frequency', 'Urban area indicator', 'Known HIV status',
       'Wealth quintile', 'tested HIV at pregnancy',
       'Delivered 12 months preceeding survey',
       'Self-reported ARV status during pregnancy', 'Breastfed last child',
       'Sexual intercourse past 12 months', 'Ever had sexual intercourse',
       'Bought/sold sex in the past 12 months',
       'Condom was used at last paid sex in past 12 months',
       'last sex partner relations past 12 months',
       'LAg: recent/long term infection', 'CD4 category', 'On ART',
       'ART initiated past 12 months those ever on ART',
       'Whether ARVs detected']      
F_X_train_ready_selected = F_X_train_ready[F_selected_features]
M_X_test_ready_selected = M_X_test_ready[M_selected_features]
F_X_test_ready_selected = F_X_test_ready[F_selected_features]
CM_test_ready_selected = CM_test_ready[M_selected_features]
CF_test_ready_selected = CF_test_ready[F_selected_features]
# Set up a dict with values to test for each parameter/argument in the model object
# stratified k-fold and model initialization
kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=random_seed)
scoring = ['f1', 'recall', 'precision']


model = lgb.LGBMClassifier()
param_grid = {
    'n_estimators': [400, 700, 1000],
    'colsample_bytree': [0.7, 0.8],
    'max_depth': [15,20,25],
    'num_leaves': [50, 100, 200],
    'reg_alpha': [1.1, 1.2, 1.3],
    'reg_lambda': [1.1, 1.2, 1.3],
    'min_split_gain': [0.3, 0.4],
    'subsample': [0.7, 0.8, 0.9],
    'subsample_freq': [20]
}



M_gm_cv = GridSearchCV(model, param_grid, scoring=scoring,
                     cv=kfold,
                     n_jobs=-1,
                     random_seed = random_seed,
                      verbose=2,
                    refit='f1')
F_gm_cv = GridSearchCV(model, param_grid, scoring=scoring,
                     cv=kfold,
                     n_jobs=-1,
                     random_seed = random_seed,
                      verbose=2,
                    refit='f1')

M_rc = M_gm_cv.fit(M_X_train_ready_selected,M_Y_train_ready)
dump(M_rc, "M_gam\M_rc_fit_sample_data.joblib")
F_rc = F_gm_cv.fit(F_X_train_ready_selected,F_Y_train_ready)
dump(F_rc, "F_gam\F_rc_fit_sample_data.joblib")

In [5]:
from matplotlib import pyplot
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve, auc
import os
import pandas as pd
import numpy as np
import pickle
from joblib import dump, load

from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import f1_score, recall_score, precision_score
from xgboost import XGBClassifier
from matplotlib import pyplot
import shap
from sklearn.metrics import confusion_matrix, f1_score, recall_score, precision_score, precision_recall_curve, auc


from sklearn.model_selection import cross_val_score, KFold
from sklearn.ensemble import RandomForestClassifier
# from sklearn.metrics import r2_score

# import matplotlib.pyplot as plt
# %matplotlib inline


random_seed = 5

# set working dFectory
working_dFectory = r"/home/dr/phia data"
os.chdF(working_dFectory)
M_X_train_ready = pickle.load(open("Train_samples\M_X_train_ready.pkl", 'rb'))
M_Y_train_ready = pickle.load(open("Train_samples\M_Y_train.pkl", 'rb'))
M_X_train_ready = pd.concat(M_X_train_ready.values(), join='inner', ignore_index=True)
M_Y_train_ready = pd.concat(M_Y_train_ready.values(), join='inner', ignore_index=True)

F_X_train_ready = pickle.load(open("Train_samples\F_X_train_ready.pkl", 'rb'))
F_Y_train_ready = pickle.load(open("Train_samples\F_Y_train.pkl", 'rb'))
F_X_train_ready = pd.concat(F_X_train_ready.values(), join='inner', ignore_index=True)
F_Y_train_ready = pd.concat(F_Y_train_ready.values(), join='inner', ignore_index=True)

M_X_test_ready = pickle.load(open("Test_samples\M_X_test_ready.pkl", 'rb'))
M_Y_test_ready = pickle.load(open("Test_samples\M_Y_test.pkl", 'rb'))
M_X_test_ready = pd.concat(M_X_test_ready.values(), join='inner', ignore_index=True)
M_Y_test_ready = pd.concat(M_Y_test_ready.values(), join='inner', ignore_index=True)

F_X_test_ready = pickle.load(open("Test_samples\F_X_test_ready.pkl", 'rb'))
F_Y_test_ready = pickle.load(open("Test_samples\F_Y_test.pkl", 'rb'))
F_X_test_ready = pd.concat(F_X_test_ready.values(), join='inner', ignore_index=True)
F_Y_test_ready = pd.concat(F_Y_test_ready.values(), join='inner', ignore_index=True)



CM_test_ready = pickle.load(open("Left_one_out_samples\CM_test_ready.pkl", 'rb'))
CM_Y_test_ready = pickle.load(open("Left_one_out_samples\CM_Y_test.pkl", 'rb'))
CM_test_ready = pd.concat(CM_test_ready.values(), join='inner', ignore_index=True)
CM_Y_test_ready = pd.concat(CM_Y_test_ready.values(), join='inner', ignore_index=True)


CF_test_ready = pickle.load(open("Left_one_out_samples\CF_test_ready.pkl", 'rb'))
CF_Y_test_ready = pickle.load(open("Left_one_out_samples\CF_Y_test.pkl", 'rb'))
CF_test_ready = pd.concat(CF_test_ready.values(), join='inner', ignore_index=True)
CF_Y_test_ready = pd.concat(CF_Y_test_ready.values(), join='inner', ignore_index=True)


M_selected_features = ['Age', 'Relationship with family head','Respondent live in household',
                        'Known HIV status','Bought/sold sex in the past 12 months',
                        'Condom was used at last paid sex in past 12 months','LAg: recent/long term infection',
                        'CD4 category','on ART','ART initiated past 12 months those ever on ART',
                        'last sex partner relations past 12 months','Sick to work last three months',
                        'Ever attended school','Ever enrolled in school','Highest level of education',
                        'Highest grade at that school level', 'Work for payment in last 12 months',
                        'Ever married/lived together','Avoiding pregnancy','CFcumcision status',
                        'Age at fFst sex', 'Ever sought TB treatment','Alcohol drink frequency',
                        'Urban area indicator','Wealth quintile', 'Sexual intercourse in the past 12 months',
                        'Ever had sexual intercourse'] 
M_X_train_ready_selected = M_X_train_ready[M_selected_features]
F_selected_features = ['Age', 'Relationship with family head', 'Respondent live in household',
       'Sick to work last three months', 'Ever attended school',
       'Ever enrolled in school', 'Highest level of education',
       'Highest grade at that school level',
       'Work for payment in last 12 months', 'Ever married/lived together',
       'Number of pregnancies',
       'Ever had a pregnancy that resulted in a live bFth',
       'No. of childen given bFth since 2012', 'Pregnacy status now',
       'Avoiding pregnancy', 'Age at fFst sex', 'Ever sought TB treatment',
       'Alcohol drink frequency', 'Urban area indicator', 'Known HIV status',
       'Wealth quintile', 'tested HIV at pregnancy',
       'Delivered 12 months preceeding survey',
       'Self-reported ARV status during pregnancy', 'Breastfed last child',
       'Sexual intercourse past 12 months', 'Ever had sexual intercourse',
       'Bought/sold sex in the past 12 months',
       'Condom was used at last paid sex in past 12 months',
       'last sex partner relations past 12 months',
       'LAg: recent/long term infection', 'CD4 category', 'On ART',
       'ART initiated past 12 months those ever on ART',
       'Whether ARVs detected']         
     
F_X_train_ready_selected = F_X_train_ready[F_selected_features]
M_X_test_ready_selected = M_X_test_ready[M_selected_features]
F_X_test_ready_selected = F_X_test_ready[F_selected_features]
CM_test_ready_selected = CM_test_ready[M_selected_features]
CF_test_ready_selected = CF_test_ready[F_selected_features]


M_RFT_selected = load("M_gam\M_rc_fit_sample_data.joblib")
F_RFT_selected = load("F_gam\F_rc_fit_sample_data.joblib")
CM_RFT_selected = load("M_gam\M_rc_fit_sample_data.joblib")
CF_RFT_selected = load("F_gam\F_rc_fit_sample_data.joblib")

M_best_index = M_RFT_selected.best_index_
F_best_index = F_RFT_selected.best_index_
CM_best_index = CM_RFT_selected.best_index_
CF_best_index = CF_RFT_selected.best_index_

M_Y_train_selected_pred = M_RFT_selected.predict(M_X_train_ready_selected)

F_Y_train_selected_pred = F_RFT_selected.predict(F_X_train_ready_selected)

print('Confusion matrix on the  M train dataset:', confusion_matrix(M_Y_train_ready, M_Y_train_selected_pred))
print('Average F1 score on the  M train sample', "{:.1%}".format(M_RFT_selected.cv_results_['mean_test_f1'][M_best_index]))
print('CI F1 score on the M train sample', "{:.1%}".format(2*M_RFT_selected.cv_results_['std_test_f1'][M_best_index]))
print('Average Sensitivity on the M train sample', "{:.1%}".format(M_RFT_selected.cv_results_['mean_test_recall'][M_best_index]))
print('CI Sensitivity on the M train sample', "{:.1%}".format(2*M_RFT_selected.cv_results_['std_test_recall'][M_best_index]))
print('Average Positive Predictive Value on the M train sample', "{:.1%}".format(M_RFT_selected.cv_results_['mean_test_precision'][M_best_index]))
print('CI Positive Predictive Value on the M train sample', "{:.1%}".format(2*M_RFT_selected.cv_results_['std_test_precision'][M_best_index]))
print('Parameters', M_RFT_selected.best_estimator_)


print('Confusion matrix on the F train dataset:', confusion_matrix(F_Y_train_ready, F_Y_train_selected_pred))
print('Average F1 score on the F train sample', "{:.1%}".format(F_RFT_selected.cv_results_['mean_test_f1'][F_best_index]))
print('CI F1 score on the F train sample', "{:.1%}".format(2*F_RFT_selected.cv_results_['std_test_f1'][F_best_index]))
print('Average Sensitivity on the F train sample', "{:.1%}".format(F_RFT_selected.cv_results_['mean_test_recall'][F_best_index]))
print('CI Sensitivity on the F train sample', "{:.1%}".format(2*F_RFT_selected.cv_results_['std_test_recall'][F_best_index]))
print('Average Positive Predictive Value on the F train sample', "{:.1%}".format(F_RFT_selected.cv_results_['mean_test_precision'][F_best_index]))
print('CI Positive Predictive Value on the F train sample', "{:.1%}".format(2*F_RFT_selected.cv_results_['std_test_precision'][F_best_index]))
print('Parameters', F_RFT_selected.best_estimator_)




M_Y_test_selected_pred = M_RFT_selected.predict(M_X_test_ready_selected)
F_Y_test_selected_pred = F_RFT_selected.predict(F_X_test_ready_selected)
CM_Y_test_selected_pred = M_RFT_selected.predict(CM_test_ready_selected)
CF_Y_test_selected_pred = F_RFT_selected.predict(CF_test_ready_selected)

print('Confusion matrix on the M test dataset:', confusion_matrix(M_Y_test_ready, M_Y_test_selected_pred))
print('F1 score on the M test dataset:', "{:.1%}".format(f1_score(M_Y_test_ready, M_Y_test_selected_pred)))
print('Sensitivity on the M test dataset:', "{:.1%}".format(recall_score(M_Y_test_ready, M_Y_test_selected_pred)))
print('Positive Predictive Value on the M test dataset:', "{:.1%}".format(precision_score(M_Y_test_ready, M_Y_test_selected_pred)))


print('Confusion matrix on the F test dataset:', confusion_matrix(F_Y_test_ready, F_Y_test_selected_pred))
print('F1 score on the F test dataset:', "{:.1%}".format(f1_score(F_Y_test_ready, F_Y_test_selected_pred)))
print('Sensitivity on the F test dataset:', "{:.1%}".format(recall_score(F_Y_test_ready, F_Y_test_selected_pred)))
print('Positive Predictive Value on the F test dataset:', "{:.1%}".format(precision_score(F_Y_test_ready, F_Y_test_selected_pred)))

print('Confusion matrix on the CF  test dataset:', confusion_matrix(CF_Y_test_ready, CF_Y_test_selected_pred))
print('F1 score on the CF test dataset:', "{:.1%}".format(f1_score(CF_Y_test_ready, CF_Y_test_selected_pred)))
print('Sensitivity on the CF test dataset:', "{:.1%}".format(recall_score(CF_Y_test_ready, CF_Y_test_selected_pred)))
print('Positive Predictive Value on the CF test dataset:', "{:.1%}".format(precision_score(CF_Y_test_ready, CF_Y_test_selected_pred)))

print('Confusion matrix on the CM test dataset:', confusion_matrix(CM_Y_test_ready, CM_Y_test_selected_pred))
print('F1 score on the CM test dataset:', "{:.1%}".format(f1_score(CM_Y_test_ready, CM_Y_test_selected_pred)))
print('Sensitivity on the CM test dataset:', "{:.1%}".format(recall_score(CM_Y_test_ready, CM_Y_test_selected_pred)))
print('Positive Predictive Value on the CM test dataset:', "{:.1%}".format(precision_score(CM_Y_test_ready, CM_Y_test_selected_pred)))



def pr_f1_curve(model, dx_test, dy_test):
         dy_test_predict_proba = model.predict_proba(dx_test)[:,1]
         dy_test_predict = model.predict(dx_test)
         ppv_curve, sensitivity_curve, _ = precision_recall_curve(dy_test, dy_test_predict_proba)
         F1_score, AUC_score = f1_score(dy_test, dy_test_predict), auc(sensitivity_curve, ppv_curve)
         print(': F1 score = %.2f - Area under Curve = %.2f' % (F1_score, AUC_score))
         prevalence = np.sum(dy_test)/len(dy_test)
         pyplot.plot([0, 1], [prevalence, prevalence], linestyle='--', label='Random Testing')
         pyplot.plot(sensitivity_curve, ppv_curve, marker='.', label='Algorithmic Testing')
         pyplot.plot(sensitivity_curve, 2*(sensitivity_curve*ppv_curve)/(sensitivity_curve+ppv_curve), marker='', label='F1')
         pyplot.xlabel('Sensitivity')
         pyplot.ylabel('Positive Predicted Value')
         pyplot.legend(bbox_to_anchor=(0.45, 0.75))
         pyplot.show()
        
    
        
pr_f1_curve(M_RFT_selected, M_X_test_ready_selected, M_Y_test_ready)
pr_f1_curve(F_RFT_selected, F_X_test_ready_selected, F_Y_test_ready)
pr_f1_curve(CF_RFT_selected, CF_test_ready_selected, CF_Y_test_ready)
pr_f1_curve(CM_RFT_selected, CM_test_ready_selected, CM_Y_test_ready)

print('Confusion matrix on the M train dataset:', confusion_matrix(M_Y_train_ready, M_Y_train_selected_pred))
print('F1 score on the train  M dataset:', "{:.1%}".format(f1_score(M_Y_train_ready, M_Y_train_selected_pred)))
print('Sensitivity on the M train dataset:', "{:.1%}".format(recall_score(M_Y_train_ready, M_Y_train_selected_pred)))
print('Positive Predictive Value on the M train dataset:', "{:.1%}".format(precision_score(M_Y_train_ready, M_Y_train_selected_pred)))


print('Confusion matrix on the F train dataset:', confusion_matrix(F_Y_train_ready, F_Y_train_selected_pred))
print('F1 score on the F train dataset:', "{:.1%}".format(f1_score(F_Y_train_ready, F_Y_train_selected_pred)))
print('Sensitivity on the F train dataset:', "{:.1%}".format(recall_score(F_Y_train_ready, F_Y_train_selected_pred)))
print('Positive Predictive Value on the F train dataset:', "{:.1%}".format(precision_score(F_Y_train_ready, F_Y_train_selected_pred)))

pr_f1_curve(M_RFT_selected, M_X_train_ready_selected, M_Y_train_ready)
pr_f1_curve(F_RFT_selected, F_X_train_ready_selected, F_Y_train_ready)
# Explain model predictions using shapley value
# M_explainer = shap.TreeExplainer(M_en1_selected.best_estimator_)
# M_shap_values = M_explainer.shap_values(M_X_train_ready_selected)
# F_explainer = shap.TreeExplainer(F_en1_selected.best_estimator_)
# F_shap_values = F_explainer.shap_values(F_X_train_ready_selected)

# CF_explainer = shap.TreeExplainer(CF_en1_selected.best_estimator_)
# CF_shap_values = CF_explainer.shap_values(CF_test_ready_selected)

# CM_explainer = shap.TreeExplainer(CM_en1_selected.best_estimator_)
# CM_shap_values = CM_explainer.shap_values(CM_test_ready_selected)

# Plot summary_plot
# shap.summary_plot(M_shap_values, M_X_train_ready_selected, max_display=24)
# shap.summary_plot(F_shap_values, F_X_train_ready_selected, max_display=29)

# shap.summary_plot(CM_shap_values, CM_test_ready_selected, max_display=24)
# shap.summary_plot(CF_shap_values, CF_test_ready_selected, max_display=29)

ValueError: Number of features of the model must match the input. Model n_features_ is 19 and input n_features is 27 